In [2]:
from config_spot import SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, redirect_uri, username, weather_api
from spotipy.oauth2 import SpotifyClientCredentials
import openweathermapy.core as owm
import spotipy
import pprint
import sys
import pandas as pd
import spotipy.util as util
import simplejson as json

In [3]:
playlist_df = pd.read_csv('Top 50 URI.csv')
playlist_df = playlist_df.set_index('Country ')

playlist_ID_list = list(playlist_df['URI'])

playlist_df['URI'][0]

'spotify:user:spotifycharts:playlist:37i9dQZEVXbMDoHDwVN2tF'

In [4]:
util.prompt_for_user_token(username,client_id=SPOTIPY_CLIENT_ID,client_secret=SPOTIPY_CLIENT_SECRET,redirect_uri='http://localhost:8888/')

'BQAkOchT6l9Vptl9CoNVE4otpk5WGyATBfzADBcywcg_HcgSfZsySega5mOZnTBtMkFamGFftbVnZrP-4qAj3JSnXAj3VehMXjzfqpP6kVVi7kcnLZXaipN4S9rZxSHC_vxLxkc9cOpp_cs'

In [5]:
token = util.oauth2.SpotifyClientCredentials(client_id= SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET )

cache_token = token.get_access_token()
spotify = spotipy.Spotify(cache_token)

In [6]:
# Generates list containing names of ALL PLAYLISTS in charts 
# Get TOP SONG from each playlist

playlist_list = []
top_songs = []
top_artists = []
song_uri_list = []

for ID in playlist_ID_list:
    playlist = spotify.user_playlist('spotifycharts', playlist_id = ID)
    name = playlist['name']
    playlist_list.append(name)
    
    top_song = playlist['tracks']['items'][0]['track']['name']
    top_songs.append(top_song)
    
    top_artist = playlist['tracks']['items'][0]['track']['artists'][0]['name']
    top_artists.append(top_artist)
    
    song_uri = playlist['tracks']['items'][0]['track']['uri']
    song_uri_list.append(song_uri)
    
    
   

In [7]:
danceability = []
energy = []
valence = []
liveness = []
tempo = []

for uri in song_uri_list:
    features = spotify.audio_features(uri)[0]
    danceability.append(features['danceability'])
    energy.append(features['energy'])
    valence.append(features['valence'])
    liveness.append(features['liveness'])
    tempo.append(features['tempo'])

In [8]:
settings = {"units": "metric", "appid": weather_api}

current_weather_paris = owm.get_current("Paris", **settings)
current_weather_paris

{'base': 'stations',
 'clouds': {'all': 0},
 'cod': 200,
 'coord': {'lat': 48.86, 'lon': 2.35},
 'dt': 1523664000,
 'id': 2988507,
 'main': {'humidity': 87,
  'pressure': 1014,
  'temp': 8.61,
  'temp_max': 10,
  'temp_min': 6},
 'name': 'Paris',
 'sys': {'country': 'FR',
  'id': 5610,
  'message': 0.0043,
  'sunrise': 1523682122,
  'sunset': 1523731241,
  'type': 1},
 'visibility': 10000,
 'weather': [{'description': 'mist',
   'icon': '50n',
   'id': 701,
   'main': 'Mist'}],
 'wind': {'deg': 60, 'speed': 2.1}}

In [ ]:
playlist_df['Top Song']= top_songs
playlist_df['Top Artist'] = top_artists
playlist_df['Song URI'] = song_uri_list
playlist_df['Danceability'] = danceability
playlist_df['Energy'] = energy
playlist_df['Liveness'] = liveness
playlist_df['Valence'] = valence
playlist_df['Tempo'] = tempo

playlist_df